In [1]:
import pandas as pd
import re 
from tqdm import tqdm

DATA_PATH = 'data/'

# Preprocess the two book dataframes 
- Load dataframes ('https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home', 'https://github.com/malcolmosh/goodbooks-10k-extended/blob/master/README.md')
- Remove faulty elements of dataframe without the descriptions 
- Remove "additions" to titles in the description dataframes - Example: title (series #1) -> title
- Make a new dataframe that contains "Book ID" (from the dataframe without descriptions), "Book Title" (-||-), "genre" (description dataframe), "description" (description dataframe)
- Store this dataframe

In [29]:
# Load dataframes
descriptions_df = pd.read_csv(DATA_PATH + "books_descriptions.csv")
book_ID_df = pd.read_json(DATA_PATH + 'goodreads_book_works.json', lines=True)

# Remove columns that are not needed in book_ID_df 
book_ID_df = book_ID_df.filter(items=["best_book_id", "original_title", "reviews_count"])


In [25]:
# List amount of books are in each dataframe 
print(f"Amount of books in descriptions_df: {len(descriptions_df)}")
print(f"Amount of books in book_ID_df: {len(book_ID_df)}")

Amount of books in descriptions_df: 10000
Amount of books in book_ID_df: 1521962


In [30]:
# Remove faulty elements from the book_ID_df
book_ID_df = book_ID_df[book_ID_df['original_title'] != '']

print(f"Amount of books in book_ID_df: {len(book_ID_df)}")

Amount of books in book_ID_df: 646906


In [31]:
# Remove "additions" to titles in the descriptions_df
for i, row in tqdm(descriptions_df.iterrows()):
    original_title = row["title"]
    new_title = re.sub(r'\((.*)', '', original_title)
    descriptions_df.at[i, "title"] = new_title.strip()

# Check if it worked -> it did
# descriptions_df.head()

10000it [00:00, 20264.09it/s]


In [32]:
# Lower case all titles to not have confusion in this manner 
descriptions_df['title'] = descriptions_df['title'].str.lower()
book_ID_df['original_title'] = book_ID_df['original_title'].str.lower()

In [33]:
# Find corresponding indexes to merge the dataframes
titles_not_found = []
book_df = pd.DataFrame(columns=['book_id', 'title', 'description', 'genres'])

for i, row in tqdm(descriptions_df.iterrows()):
    title = row['title']
    # Check if the title is in book_ID_df, else append it to the titles_not_found list
    if title in book_ID_df['original_title'].values:
        # Get all rows that have the a matching title as the current row
        temp_df = book_ID_df[book_ID_df['original_title'] == title]
        
        # Get the book id of the book  with the highest amount of reviews
        book_id = temp_df['best_book_id'][temp_df['reviews_count'].idxmax()]
        descriptions = row['description']
        genres = row['genres']
        book_df = book_df.append({'book_id': book_id, 'title': title, 'description': descriptions, 'genres': genres}, ignore_index=True)
    else: 
        titles_not_found.append(title)

# print the amount of elements that are not found
print(f"Out of the 10000 titles {len(titles_not_found)} are not found in the book_ID_df")

10000it [09:56, 16.77it/s]

Out of the 10000 titles 2082 are not found in the book_ID_df


In [34]:
# Save book_df to a csv 
book_df.to_csv(DATA_PATH + 'book_df.csv', index=False)

In [35]:
# TODO - do this better somehow? 
# Ideas, remove all special characters (but then also " ") 
# Convert all to tokens, then make the inner product of the tokens, normalize and take the highest value. 


# # Find corresponding indexes to merge the dataframes 
# matching_idxs = []

# count = 0
# for i, title in enumerate(descriptions_df['title']):
#     title_found_count = 0
#     if title in book_ID_df['original_title'].values:
#         pass
#         # print(f"Found match: '{title}', at index {i}")
        
#         # #Save pair of matching indexes
#         # matching_idxs.append((i, j))
#     else:
        
#         title_found = False
#         first_match = False
#         while True:
#             #Check all original titles for a match
#             for original_title in book_ID_df['original_title']:
#                 if title == original_title[:len(title)]:
#                     title_found = True
#                     if not first_match:
#                         shortest_title = original_title
#                     else:
#                         if len(original_title) < len(shortest_title): 
#                             shortest_title = original_title
#             if title_found:
#                 #print(f"Found match: '{title}' in '{shortest_title}'")
#                 break
#             title = title[:-1]
#             if not len(title):
#                 break


Found match: 'harry potter and the sorcerer's stone' in 'harry potter and the sorcerer's stone: selected themes from the motion picture : french horn solo, duet, trio (instrumental series)'
Found match: 'the girl with the dragon tattoo' in 'the girl with the dragon tattoo, vol. 2'
Found match: 'the lion, the witch, and the ' in 'the lion, the witch, and the bible:  good and evil in the classic tales of c.s. lewis'
Found match: 'eat, pray, love' in 'eat, pray, love: one woman's search for everything across italy, india and indonesia'
Found match: 'the golden compass' in 'the golden compass movie storybook'
Found match: 'slaughterhouse-five' in 'slaughterhouse-five / god bless you, mr rosewater / breakfast of champions'
Found match: 'bridget jones's diary' in 'bridget jones's diary: music from the motion picture'
Found match: 'the little prince' in 'the little princess'
Found match: 'one hundred years of solitude' in 'one hundred years of solitude [cliffs notes study] (notes)'
Found matc

KeyboardInterrupt: 

# Preprocess the shelves 
- Use "book_id_map.csv" to find the books we use (ids) and store "new_ids" (the ids we can use to find the relevant shelfs)
- Drop all rows in "goodreads_interactions.csv" that have different ids than "new_ids". 
- Store this dataset.  

In [35]:
# Load dataframes
book_id_map_df = pd.read_csv(DATA_PATH + "book_id_map.csv")
book_df = pd.read_csv(DATA_PATH + "book_df.csv")

In [36]:
# Create map from book_id to book_id_csv
book_id_map = {book_id_map_df['book_id'][i]: book_id_map_df['book_id_csv'][i] for i in range(len(book_id_map_df))}

# Get the id's we are interested in 
for i in range(len(book_df)):
    book_df["book_id"][i] = book_id_map[book_df["book_id"][i]]


C:\Users\david\AppData\Local\Temp/ipykernel_15324/3212419484.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_df["book_id"][i] = book_id_map[book_df["book_id"][i]]


KeyError: 6404663

In [ ]:
# Load the shelves dataframe 
shelves_df = pd.read_csv(DATA_PATH + "goodreads_interactions.csv")

# Check how many books are on the shelves of ALL users combined 
len(shelves_df)

228648342

In [ ]:
# Remove books on the shelves that are not in the book_df
shelves_df = shelves_df[shelves_df['book_id'].isin(book_df['book_id'].tolist())]

# Save the new shelves_df 
shelves_df.to_csv(DATA_PATH + 'shelves_df.csv', index=False)

In [ ]:
# Check how many books are on the shelves of ALL users combined - after removal of books not in book_df
len(shelves_df)

1871767

In [ ]:
# Save the book_df dataframe with the index change 
book_df.to_csv(DATA_PATH + 'book_matching_ids_df.csv', index=False)

In [ ]:
len(set(shelves_df["book_id"].tolist()))

2360650